# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [9]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import pickle

nltk.download(['wordnet', 'punkt', 'stopwords'])



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
# load data from database
engine = create_engine('sqlite:///disaster_messages_database.db')
df = pd.read_sql_table('disaster_messages_table', engine)
X = df['message'] # Message Column
Y = df.iloc[:, 4:] # Classification label

### 2. Write a tokenization function to process your text data

In [11]:
def tokenize(text):
    
     """
    Function: spilit text into words and return the root form of the words
    Args:
      text(str): the message
    Return:
      lemm(list of str): a list of the root form of the message words
    """
        
    # Normalize text
     text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize text
     words = word_tokenize(text)
    
    # Remove stop words
     stop = stopwords.words("english")
     words = [t for t in words if t not in stop]
    
    # Lemmatization
     lemm = [WordNetLemmatizer().lemmatize(w) for w in words]
    
     return lemm

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
# Create a pipeline
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
# Spiliting data
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# Fit the pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
# Predicting
y_pred = pipeline.predict(X_test)

In [15]:
# Testing the model
# Printing the classification report for each label

i = 0
for col in y_test:
        print('Feature {}: {}'.format(i+1, col))
        print(classification_report(y_test[col], y_pred[:, i]))
        i = i + 1

Feature 1: related
             precision    recall  f1-score   support

          0       0.61      0.48      0.54      1530
          1       0.85      0.90      0.87      4964
          2       0.36      0.33      0.34        60

avg / total       0.79      0.80      0.79      6554

Feature 2: request
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5427
          1       0.81      0.45      0.58      1127

avg / total       0.88      0.89      0.87      6554

Feature 3: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6529
          1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      6554

Feature 4: aid_related
             precision    recall  f1-score   support

          0       0.75      0.84      0.79      3877
          1       0.72      0.60      0.66      2677

avg / total       0.74      0.74      0.74      6554

Feature 5

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [16]:
accuracy = (y_pred == y_test.values).mean()
print('The model accuracy is {:.3f}'.format(accuracy))

The model accuracy is 0.945


### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
# Show parameters for the pipline
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fa328669598>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [18]:
# Create Grid search parameters
parameters = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [10, 20]
}

cv = GridSearchCV(pipeline, param_grid = parameters)
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
# Fit the tuned model
cv.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
# Predicting
y_pred = cv.predict(X_test)

# Testing the model
# Printing the classification report for each label
print('The results of the tuned model')
i = 0
for col in y_test:
        print('Feature {}: {}'.format(i+1, col))
        print(classification_report(y_test[col], y_pred[:, i]))
        i = i + 1
        
accuracy = (y_pred == y_test.values).mean()
print('The tuned model accuracy is {:.3f}'.format(accuracy))

In [ ]:
# Showing the best parameters
print("Best Parameters:", cv.best_params_)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
# Creating a new pipline to improve the model
pipeline2 = Pipeline([
        ('tfidfvect', TfidfVectorizer()),
        ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [15]:
# Fit the improved model
pipeline2.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidfvect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tru...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [16]:
# Predicting
y_pred = pipeline2.predict(X_test)

# Testing the model
# Printing the classification report for each label
print('The results of the improved model')
i = 0
for col in y_test:
        print('Feature {}: {}'.format(i+1, col))
        print(classification_report(y_test[col], y_pred[:, i]))
        i = i + 1
        
accuracy = (y_pred == y_test.values).mean()
print('The improved model accuracy is {:.3f}'.format(accuracy))

The results of the improved model
Feature 1: related
             precision    recall  f1-score   support

          0       0.58      0.25      0.35      1576
          1       0.80      0.94      0.86      4944
          2       0.08      0.06      0.07        34

avg / total       0.74      0.77      0.74      6554

Feature 2: request
             precision    recall  f1-score   support

          0       0.91      0.97      0.94      5437
          1       0.76      0.52      0.62      1117

avg / total       0.88      0.89      0.88      6554

Feature 3: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.11      0.03      0.05        29

avg / total       0.99      0.99      0.99      6554

Feature 4: aid_related
             precision    recall  f1-score   support

          0       0.75      0.87      0.81      3851
          1       0.77      0.59      0.67      2703

avg / total       0.76      0

In [17]:
# Show parameters for the pipline
pipeline2.get_params()

{'memory': None,
 'steps': [('tfidfvect',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
           stop_words=None, strip_accents=None, sublinear_tf=False,
           token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
           vocabulary=None)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'tfidfvect': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), norm='l2', preprocessor=None, smooth

In [20]:
# Using grid search
# Create Grid search parameters
parameters2 = {
        'tfidfvect__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 60, 70]
}

cv2 = GridSearchCV(pipeline2, param_grid = parameters2)

# Fit the  model
cv2.fit(X_train, y_train)

# Predicting
y_pred = cv2.predict(X_test)

# Testing the model
# Printing the classification report for each label
print('The results of the model')
i = 0
for col in y_test:
        print('Feature {}: {}'.format(i+1, col))
        print(classification_report(y_test[col], y_pred[:, i]))
        i = i + 1
        
accuracy = (y_pred == y_test.values).mean()
print('The model accuracy is {:.3f}'.format(accuracy))

# Showing the best parameters
print("Best Parameters:", cv2.best_params_)


The results of the model
Feature 1: related
             precision    recall  f1-score   support

          0       0.59      0.16      0.26      1576
          1       0.78      0.96      0.86      4944
          2       0.17      0.09      0.12        34

avg / total       0.73      0.77      0.71      6554

Feature 2: request
             precision    recall  f1-score   support

          0       0.91      0.96      0.94      5437
          1       0.75      0.54      0.63      1117

avg / total       0.88      0.89      0.88      6554

Feature 3: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.07      0.03      0.05        29

avg / total       0.99      0.99      0.99      6554

Feature 4: aid_related
             precision    recall  f1-score   support

          0       0.76      0.86      0.81      3851
          1       0.76      0.61      0.68      2703

avg / total       0.76      0.76      

### 9. Export your model as a pickle file

In [21]:
# Create a pickle file for the model
file_name = 'classifier.pkl'
with open (file_name, 'wb') as f:
    pickle.dump(cv, f) 

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.